<a id='1'></a>
## 1 - Loading the libraries

---

Load các library cần thiết

In [1]:
import joblib
import weaviate
from weaviate.classes.query import (
    Filter, 
    Rerank
)

In [2]:
import flask_app
import weaviate_server
from utils import (
    generate_with_single_input,
    print_object_properties,
    display_widget
)
import unittests

 * Serving Flask app 'flask_app'
 * Debug mode: off


In [3]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
import json


<a id='2'></a>
## 2 - Setting up the Weaviate Client and loading the data

---

Thiết lập Weaviate client và load data từ bộ dữ liệu [BBC news dataset](https://www.kaggle.com/datasets/gpreda/bbc-news) adapted từ Kaggle.

<a id='2-1'></a>
### 2.1 Loading the Weaviate Client

Let's connect the Weaviate client to begin working with the Weaviate API. The server is already running on the backend.

In [4]:
client = weaviate.connect_to_local(port=8079, grpc_port=50050)

<a id='2-2'></a>
### 2.2 Loading the data

Load data. Bộ dữ liệu được cấu trúc với các fields sau:

- **`title`**: Tiêu đề của bài báo.
- **`pubDate`**: Ngày và giờ xuất bản của bài báo.
- **`guid`**: Mã định danh duy nhất (unique identifier) cho bài báo, thường được dùng để listing.
- **`link`**: Đường dẫn URL để truy cập bài báo đầy đủ trực tuyến.
- **`description`**: Một đoạn tóm tắt ngắn hoặc nội dung giới thiệu (teaser) của bài báo.
- **`article_content`**: Toàn bộ nội dung văn bản của bài báo, cung cấp thông tin chi tiết.

In [ ]:
bbc_data = joblib.load('data/bbc_data.joblib')

In [ ]:
print_object_properties(bbc_data[0])

<a id='3'></a>
## 3 - Loading the Collection

---

Load bộ collection có chứa BBC News dataset.

In [ ]:
collection = client.collections.get("bbc_collection")

In [ ]:
print(f"The number of elements in the collection is: {len(collection)}")

In [ ]:
object = collection.query.fetch_objects(limit = 1, include_vector = True).objects[0]
print("Printing the properties (some will be truncated due to size)")
print_object_properties(object.properties)
print("Vector: (truncated)",object.vector['main_vector'][0:15])
print("Vector length: ", len(object.vector['main_vector']))

<a id='ex01'></a>

<a id='3-1'></a>
### 3.1 Metadata filtering

In [ ]:
def filter_by_metadata(metadata_property: str, 
                       values: list[str], 
                       collection: "weaviate.collections.collection.sync.Collection" , 
                       limit: int = 5) -> list:
    """
    Retrieves objects từ một collection được chỉ định dựa trên các tiêu chí metadata filtering.

    Hàm này truy vấn một collection trong client được chỉ định để lấy các objects khớp với các tiêu chí metadata nhất định. Nó sử dụng một filter để tìm các objects có 'property' được chỉ định chứa bất kỳ giá trị nào trong số các 'values' đã cho. Số lượng objects được truy xuất bị giới hạn bởi tham số 'limit'.

    Args:
    metadata_property (str): Tên của metadata property dùng để lọc.
    values (List[str]): Một danh sách các giá trị để đối chiếu với property đã chỉ định.
    collection_name (weaviate.collections.collection.sync.Collection): Collection để thực hiện truy vấn.
    limit (int, optional): Số lượng objects tối đa được truy xuất. Mặc định là 5.

    Returns:
    List[Object]: Một danh sách các objects từ collection khớp với các tiêu chí lọc.
    """
    
    # Retrieve using collection.query.fetch_objects
    
    response = collection.query.fetch_objects(
        filters = Filter.by_property(metadata_property).contains_any(values),
        limit=limit)

    
    response_objects = [x.properties for x in response.objects]
    
    return response_objects

In [ ]:
# Example
res = filter_by_metadata('title', ['Taylor Swift'], collection, limit = 2)
for x in res:
    print_object_properties(x)

In [ ]:
# Test your solution!
unittests.test_filter_by_metadata(filter_by_metadata, client)

<a id='ex02'></a>

<a id='3-2'></a>
### 3.2 Semantic search

In [ ]:
def semantic_search_retrieve(query: str,
                             collection: "weaviate.collections.collection.sync.Collection" , 
                             top_k: int = 5) -> list:
    """
    Thực hiện một semantic search trên một collection và truy xuất các chunks liên quan nhất.

    Hàm này thực thi một truy vấn semantic search trên một collection được chỉ định để tìm các text chunks có độ liên quan cao nhất với đầu vào 'query'. Phép tra cứu truy xuất một số lượng giới hạn các matching objects hàng đầu, được chỉ định bởi 'top_k'. Hàm trả về thuộc tính 'chunk' của mỗi top matching objects đó.

    Args:
    query (str): Truy vấn tìm kiếm được sử dụng để tìm các text chunks liên quan.
    collection (weaviate.collections.collection.sync.Collection): Collection nơi phép semantic search được thực hiện.
    top_k (int, optional): Số lượng các relevant objects hàng đầu cần truy xuất. Mặc định là 5.

    Returns:
    List[str]: Một danh sách các text chunks có độ liên quan cao nhất với truy vấn đã cho.
    """

    # Retrieve using collection.query.near_text
    response = collection.query.near_text(query=query, limit=top_k)
    
    
    response_objects = [x.properties for x in response.objects]
    
    return response_objects

In [ ]:
# Let's have an example!
print_object_properties(semantic_search_retrieve(query = 'Tell me about the last Taylor Swift show', collection = collection, top_k = 2))

In [ ]:
unittests.test_semantic_search_retrieve(semantic_search_retrieve, client)

<a id='ex03'></a>

<a id='3-3'></a>
### 3.3 BM25 Serach

In [ ]:
def bm25_retrieve(query: str, 
                  collection: "weaviate.collections.collection.sync.Collection" , 
                  top_k: int = 5) -> list:
    """
    Thực hiện một BM25 search trên một collection và truy xuất các chunks liên quan nhất.
    Hàm này thực thi một truy vấn tìm kiếm dựa trên BM25 trên một collection được chỉ định để xác định các text chunks có độ liên quan cao nhất với 'query' được cung cấp. Nó truy xuất một số lượng giới hạn các matching objects hàng đầu, được chỉ định bởi 'top_k', và trả về thuộc tính 'chunk' của các objects này.

    Args:
    query (str): Truy vấn tìm kiếm được sử dụng để tìm các text chunks liên quan.
    collection (weaviate.collections.collection.sync.Collection): Collection nơi phép BM25 search được thực hiện.
    top_k (int, optional): Số lượng các relevant objects hàng đầu cần truy xuất. Mặc định là 5.

    Returns:
    List[str]: Một danh sách các text chunks có độ liên quan cao nhất với truy vấn đã cho.
    """

    # Retrieve using collection.query.bm25
    response = collection.query.bm25(
        query=query,
        limit=top_k
    )
    
    response_objects = [x.properties for x in response.objects]
    return response_objects 

In [ ]:
print_object_properties(bm25_retrieve('Tell me about the last Taylor Swift show', collection, top_k = 2))

In [ ]:
unittests.test_bm25_retrieve(bm25_retrieve, client)

<a id='ex04'></a>

<a id='3-4'></a>
### 3.4 Hybrid search

Triển khai một hệ thống retrieval sử dụng Reciprocal Rank Fusion (RRF) thông qua Weaviate API. Để đạt được điều này, cần sử dụng phương thức `collection.query.hybrid`.

In [ ]:
def hybrid_retrieve(query: str, 
                    collection: "weaviate.collections.collection.sync.Collection" , 
                    alpha: float = 0.5,
                    top_k: int = 5
                   ) -> list:
    """
    Thực hiện một hybrid search trên một collection và truy xuất các chunks liên quan nhất.
    Hàm này thực thi một truy vấn hybrid search kết hợp giữa semantic vector search và tìm kiếm dựa trên từ khóa truyền thống (keyword-based search) trên một collection được chỉ định để tìm các text chunks có độ liên quan cao nhất với đầu vào 'query'. Độ liên quan của kết quả bị ảnh hưởng bởi tham số 'alpha', giúp cân bằng trọng số giữa khớp vector (vector matches) và khớp từ khóa (keyword matches). Nó truy xuất một số lượng giới hạn các matching objects hàng đầu, được chỉ định bởi 'top_k', và trả về thuộc tính 'chunk' của các objects này.

    Args:
    query (str): Truy vấn tìm kiếm được sử dụng để tìm các text chunks liên quan.
    collection (weaviate.collections.collection.sync.Collection): Collection nơi phép hybrid search được thực hiện.
    alpha (float, optional): Một hệ số trọng số giúp cân bằng sự đóng góp của semantic matches và keyword matches. Mặc định là 0.5.
    top_k (int, optional): Số lượng các relevant objects hàng đầu cần truy xuất. Mặc định là 5.

    Returns:
    List[str]: Một danh sách các text chunks có độ liên quan cao nhất với truy vấn đã cho.
    """

    # Retrieve using collection.query.hybrid
    response = collection.query.hybrid(
        query=query,
        alpha=alpha,
        limit=top_k
    )

    
    response_objects = [x.properties for x in response.objects]
    
    return response_objects 

In [ ]:
print_object_properties(hybrid_retrieve('Tell me about the last Taylor Swift show', collection, top_k = 2))

### 3.5 Query Classification

In [ ]:
def classify_query_type(query: str) -> str:
    """
    Phân loại query thành 'technical' hoặc 'creative'. 
    
    - Technical: Câu hỏi về sự kiện, số liệu, thông tin cụ thể
    - Creative: Yêu cầu tổng hợp, phân tích, đưa ra ý kiến
    
    Args:
        query (str): Câu hỏi của người dùng
        
    Returns: 
        str: 'technical' hoặc 'creative'
    """
    PROMPT = f"""Analyze the following query and classify it as either 'technical' or 'creative'. 

Technical queries: 
- Ask for specific facts, dates, numbers, or events
- Request documentation or procedural information
- Seek objective, factual answers

Creative queries: 
- Ask for analysis, opinions, or interpretations
- Request summaries or comparisons
- Seek subjective or exploratory answers

Query:  {query}

Answer only 'technical' or 'creative' (one word, lowercase).
"""
    result = generate_with_single_input(PROMPT, max_tokens=5)
    label = result['content'].strip().lower()
    
    # Validate output
    if label not in ['technical', 'creative']:
        label = 'technical'  # Default fallback
    
    return label


def classify_news_category(query: str) -> str:
    """
    Phân loại query theo danh mục tin tức để tối ưu retrieval.
    
    Categories:  politics, sports, entertainment, business, technology, general
    
    Args:
        query (str): Câu hỏi của người dùng
        
    Returns: 
        str:  Danh mục tin tức
    """
    PROMPT = f"""Classify the following news query into one of these categories:
- politics:  Government, elections, international relations
- sports:  Games, athletes, tournaments
- entertainment: Movies, music, celebrities
- business: Economy, markets, companies
- technology: Tech companies, innovations, digital
- general: Other topics

Query: {query}

Answer with only ONE category name (lowercase).
"""
    result = generate_with_single_input(PROMPT, max_tokens=5)
    category = result['content'].strip().lower()
    
    valid_categories = ['politics', 'sports', 'entertainment', 'business', 'technology', 'general']
    if category not in valid_categories:
        category = 'general'
    
    return category

### 3.6 Dynamic Parameter Selection

In [ ]:
def get_llm_params_for_query(query: str) -> dict:
    """
    Tự động chọn tham số LLM dựa trên loại query.
    
    Args:
        query (str): Câu hỏi của người dùng
        
    Returns: 
        dict:  Tham số cho LLM call (temperature, top_p)
    """
    query_type = classify_query_type(query)
    
    if query_type == 'technical': 
        # Technical:  Cần chính xác, ít ngẫu nhiên
        params = {
            'temperature':  0.1,
            'top_p': 0.1,
            'description': 'Technical mode:  Low randomness for factual accuracy'
        }
    else:  # creative
        # Creative: Cho phép sáng tạo hơn
        params = {
            'temperature':  0.7,
            'top_p': 0.4,
            'description': 'Creative mode: Higher randomness for diverse responses'
        }
    
    return params


def get_retrieval_params_for_query(query:  str) -> dict:
    """
    Tự động chọn tham số retrieval dựa trên loại query.
    """
    query_type = classify_query_type(query)
    category = classify_news_category(query)
    
    # Base params - mặc định KHÔNG dùng rerank để tránh lỗi
    params = {
        'top_k': 5,
        'alpha': 0.5,
        'use_rerank': False,  # Default: không rerank
        'rerank_property': 'chunk'
    }
    
    if query_type == 'technical':
        params['top_k'] = 7
        params['alpha'] = 0.3
        # Chỉ bật rerank cho technical queries quan trọng
        # Có thể bật lại nếu muốn:  params['use_rerank'] = True
    else:  
        params['top_k'] = 5
        params['alpha'] = 0.7
    
    if category in ['politics', 'business']: 
        params['top_k'] += 2
    
    return params

### 3.7 Structured Output Schemas

In [ ]:
class NewsSource(BaseModel):
    """Schema cho một nguồn tin tức"""
    title: str = Field(description="Tiêu đề bài báo")
    url: str = Field(description="Link đến bài báo")
    published_date: str = Field(description="Ngày xuất bản")
    relevance:  str = Field(description="Mức độ liên quan:  high/medium/low")


class RAGResponse(BaseModel):
    """Schema cho response của RAG system"""
    answer: str = Field(description="Câu trả lời chi tiết cho query")
    summary: str = Field(description="Tóm tắt 1-2 câu")
    confidence: str = Field(description="Độ tin cậy: high/medium/low")
    sources: List[NewsSource] = Field(description="Danh sách các nguồn được sử dụng")
    query_type: str = Field(description="Loại query:  technical/creative")
    category: str = Field(description="Danh mục tin tức")


class FactCheckResponse(BaseModel):
    """Schema cho việc kiểm tra thông tin"""
    claim: str = Field(description="Thông tin cần kiểm tra")
    verdict:  Literal["true", "false", "partially_true", "unverifiable"] = Field(
        description="Kết luận"
    )
    evidence: List[str] = Field(description="Bằng chứng từ các nguồn")
    sources: List[NewsSource] = Field(description="Nguồn tham khảo")

### 3.8 - Reranking

Tạo một phiên bản mới của `semantic_search` cho phép thực hiện reranking các kết quả. Hàm mới này hỗ trợ việc sử dụng một query khác cho mục đích reranking hoặc thực hiện reranking dựa trên một thuộc tính tài liệu cụ thể (ví dụ: reranking chỉ sử dụng trường title).

Nhiệm vụ là phải thêm tham số `rerank` vào lệnh gọi `collection.query.near_text`.

In [ ]:
def semantic_search_with_reranking(query: str, 
                                   rerank_property: str,
                                   collection: "weaviate.collections.collection.sync.Collection" , 
                                   rerank_query: str = None,
                                   top_k: int = 5
                                   ) -> list:
    """
    Thực hiện một semantic search và thực hiện reranks các kết quả dựa trên một property được chỉ định.

    Args:
    query (str): Truy vấn tìm kiếm để thực hiện bước tìm kiếm ban đầu (initial search).
    rerank_property (str): Thuộc tính được sử dụng để reranking các kết quả tìm kiếm.
    collection (weaviate.collections.collection.sync.Collection): Collection để thực hiện tìm kiếm bên trong.
    rerank_query (str, optional): Truy vấn được sử dụng riêng cho mục đích reranking. Nếu không được cung cấp, truy vấn gốc (original query) sẽ được sử dụng để reranking.
    top_k (int, optional): Số lượng kết quả hàng đầu tối đa được trả về. Mặc định là 5.

    Returns:
    list: Một danh sách các properties từ các kết quả tìm kiếm đã được reranked, trong đó mỗi mục tương ứng với một object trong collection.
    """

    # Set the rerank_query to be the same as the query if rerank_query is not passed
    if rerank_query is None: 
        rerank_query = query 
        
    # Define the reranker with rerank_query and rerank_property
    reranker = Rerank(
        prop=rerank_property,
        query=rerank_query
    )

    # Retrieve using collection.query.near_text with the appropriate parameters (do not forget the rerank!)
    response = collection.query.near_text(
        query=query,
        limit=top_k,
        rerank=reranker
    )
    
    response_objects = [x.properties for x in response.objects]
    
    return response_objects 

In [ ]:
# Set a query
query = 'Tell me about the conflicts in Latin America'
# Get the results from a search (in this case the hybrid search)
results = semantic_search_with_reranking(query, collection = collection, top_k = 2, rerank_property = 'chunk')

In [ ]:
print_object_properties(results)

In [ ]:
# Test your function!
unittests.test_semantic_search_with_reranking(semantic_search_with_reranking, client)

<a id='4'></a>
## 4 - Incorporating the Weaviate API into our previous schema
---

Tại đây, bạn sẽ xem lại các hàm đã được sử dụng xuyên suốt để tích hợp Weaviate API vào schema hiện tại của mình. Sau khi tích hợp, bạn sẽ có thể chạy các prompts và kiểm tra hệ thống RAG system mới của mình!

<a id='4-1'></a>
### 4.1 Generating the final prompt


In [ ]:
def generate_final_prompt(
    query: str, 
    top_k: int, 
    retrieve_function:  callable,
    rerank_query: str = None, 
    rerank_property: str = None, 
    use_rerank: bool = False, 
    use_rag: bool = True,
    output_format: str = "text",
    include_classification: bool = True
) -> str:
    """
    Generates a final prompt với các tính năng prompt engineering. 
    """
    # Nếu không dùng RAG, trả về query gốc
    if not use_rag:
        return query
    
    # Phân loại query
    query_type = classify_query_type(query) if include_classification else "general"
    category = classify_news_category(query) if include_classification else "general"
    
    # Retrieve documents
    if use_rerank:
        if rerank_property is None:
            rerank_property = 'chunk'  # Default property
        # Khi cần rerank, PHẢI dùng semantic_search_with_reranking
        top_k_documents = semantic_search_with_reranking(
            query=query, 
            top_k=top_k, 
            collection=collection, 
            rerank_property=rerank_property, 
            rerank_query=rerank_query
        )
    else:
        # Dùng retrieve_function được truyền vào (hybrid, semantic, bm25)
        top_k_documents = retrieve_function(
            query=query, 
            top_k=top_k, 
            collection=collection
        )
    
    # Format documents
    formatted_data = ""
    source_list = []
    
    for i, document in enumerate(top_k_documents, 1):
        document_layout = (
            f"[Source {i}]\n"
            f"Title: {document['title']}\n"
            f"Content: {document['chunk']}\n"
            f"Published: {document['pubDate']}\n"
            f"URL:  {document['link']}\n"
        )
        formatted_data += document_layout + "\n"
        
        source_list.append({
            "title":  document['title'],
            "url": document['link'],
            "published_date": str(document['pubDate'])
        })
    
    # Xây dựng prompt dựa trên output format
    if output_format == "json":
        prompt = _build_json_prompt(query, query_type, category, formatted_data, source_list)
    else:
        prompt = _build_text_prompt(query, query_type, category, formatted_data)
    
    return prompt


def _build_text_prompt(query: str, query_type: str, category: str, formatted_data: str) -> str:
    """Xây dựng prompt cho text output"""
    
    # Điều chỉnh instructions dựa trên query type
    if query_type == "technical":
        style_instruction = """
- Be precise and factual
- Include specific dates, numbers, and names when available
- Cite sources explicitly
- Avoid speculation
"""
    else:  # creative
        style_instruction = """
- Provide analysis and context
- Connect different pieces of information
- Offer insights and implications
- Use engaging language
"""
    
    prompt = f"""You are a news analyst assistant. Answer the user's query using the provided news sources.

**Query Type**:  {query_type}
**Category**: {category}

**Instructions**:
{style_instruction}
- Always cite your sources using [Source N] format
- If information is uncertain, acknowledge it
- Structure your response clearly

**User Query**: {query}

**Available News Sources**:
{formatted_data}

**Your Response**:
"""
    return prompt


def _build_json_prompt(
    query: str, 
    query_type: str, 
    category: str, 
    formatted_data: str, 
    source_list: list
) -> str:
    """Xây dựng prompt cho JSON output"""
    
    json_schema = RAGResponse. model_json_schema()
    
    prompt = f"""You are a news analyst assistant. Answer the user's query and return a structured JSON response.

**Query Type**: {query_type}
**Category**: {category}

**User Query**: {query}

**Available News Sources**:
{formatted_data}

**Instructions**:
- Respond ONLY with valid JSON matching the schema below
- Fill in all required fields
- Be accurate and cite sources properly

**JSON Schema**:
{json. dumps(json_schema, indent=2)}

**Your JSON Response**:
"""
    return prompt

In [ ]:
prompt = generate_final_prompt("Tell me the economic situation of the US in 2024.", top_k = 5, retrieve_function = semantic_search_retrieve, use_rerank = False, rerank_property = 'title')

In [ ]:
print(prompt)

<a id='4-2'></a>
### 4.2 LLM call

Hãy xem lại hàm llm_call, hiện đã được adapted.

In [ ]:
def llm_call(
    query: str, 
    retrieve_function: callable = None, 
    top_k: int = None,
    use_rag: bool = True, 
    use_rerank: bool = None,
    rerank_property: str = None, 
    rerank_query: str = None,
    output_format: str = "text",
    auto_tune: bool = True
) -> dict:
    """
    Enhanced LLM call với prompt engineering features.
    """
    # Default retrieve function
    if retrieve_function is None:  
        retrieve_function = hybrid_retrieve
    
    # Auto-tune parameters nếu được bật
    if auto_tune:
        llm_params = get_llm_params_for_query(query)
        retrieval_params = get_retrieval_params_for_query(query)
        
        if top_k is None:  
            top_k = retrieval_params['top_k']
        if use_rerank is None:  
            use_rerank = retrieval_params['use_rerank']
        if rerank_property is None and use_rerank: 
            rerank_property = retrieval_params. get('rerank_property', 'chunk')
    else:
        llm_params = {'temperature': 0.5, 'top_p': 0.5}
        if top_k is None: 
            top_k = 5
        if use_rerank is None:  
            use_rerank = False
    
    # Lấy thông tin classification
    query_type = classify_query_type(query)
    category = classify_news_category(query)
    
    # Generate prompt
    # LƯU Ý: Khi use_rerank=True, generate_final_prompt sẽ tự động
    # sử dụng semantic_search_with_reranking thay vì retrieve_function
    PROMPT = generate_final_prompt(
        query=query, 
        top_k=top_k, 
        retrieve_function=retrieve_function,  # Chỉ dùng khi use_rerank=False
        use_rag=use_rag, 
        use_rerank=use_rerank, 
        rerank_property=rerank_property, 
        rerank_query=rerank_query,
        output_format=output_format,
        include_classification=True
    )
    
    # Gọi LLM với params đã điều chỉnh
    if output_format == "json": 
        response = generate_with_multiple_input(
            messages=[
                {"role": "system", "content": "You are a helpful news analyst.  Respond only in valid JSON. "},
                {"role": "user", "content":  PROMPT}
            ],
            response_format={
                "type":  "json_schema",
                "schema": RAGResponse.model_json_schema()
            }
        )
        content = response['content']
        
        try:  
            parsed_content = json.loads(content)
        except json.JSONDecodeError:
            parsed_content = {"error": "Failed to parse JSON", "raw":  content}
    else:
        # Text output - kiểm tra xem generate_with_single_input có hỗ trợ params không
        try:
            response = generate_with_single_input(
                PROMPT,
                temperature=llm_params. get('temperature', 0.5),
                top_p=llm_params. get('top_p', 0.5)
            )
        except TypeError:
            # Fallback nếu function không hỗ trợ temperature/top_p
            response = generate_with_single_input(PROMPT)
        
        content = response['content']
        parsed_content = None
    
    return {
        'content': content,
        'parsed_json': parsed_content,
        'query_type': query_type,
        'category': category,
        'params_used': {
            'llm':  llm_params,
            'retrieval': {
                'top_k': top_k,
                'use_rerank': use_rerank,
                'rerank_property': rerank_property
            }
        }
    }

In [ ]:
query = "Tell me about United States and Brazil's relationship over the course of 2024. Provide links for the resources you use in the answer."

In [ ]:
# Result with reranked results
print(llm_call(query = query, 
               top_k = 5, 
               retrieve_function = hybrid_retrieve, 
               ))

### 4.2.1 WRAPPER Cho DISPLAY_WIDGET

In [ ]:
# Bước 1: Lưu phiên bản advanced với tên khác
llm_call_advanced = llm_call

# Bước 2: Ghi đè llm_call để trả về string (tương thích widget)
def llm_call(
    query: str, 
    retrieve_function:  callable = None, 
    top_k: int = 5, 
    use_rag: bool = True, 
    use_rerank: bool = False, 
    rerank_property: str = None, 
    rerank_query:  str = None
) -> str:
    """
    Phiên bản llm_call tương thích với display_widget.
    Tự động áp dụng prompt engineering và trả về string. 
    
    Để sử dụng phiên bản đầy đủ với dict output, dùng:  llm_call_advanced()
    """
    try:
        result = llm_call_advanced(
            query=query,
            retrieve_function=retrieve_function,
            top_k=top_k if top_k else 5,
            use_rag=use_rag,
            use_rerank=use_rerank,
            rerank_property=rerank_property,
            rerank_query=rerank_query,
            output_format="text",
            auto_tune=True
        )
        
        # Tạo header với metadata
        header = f"""
> **Query Analysis**:  `{result['query_type']}` | **Category**: `{result['category']}` | **Temp**: `{result['params_used']['llm']['temperature']}`

---

"""
        return header + result['content']
    
    except Exception as e: 
        # Fallback nếu có lỗi
        print(f"Warning: Advanced features failed ({e}), using basic mode")
        
        if retrieve_function is None: 
            retrieve_function = hybrid_retrieve
            
        if use_rerank and rerank_property: 
            docs = semantic_search_with_reranking(
                query=query, 
                collection=collection, 
                top_k=top_k, 
                rerank_property=rerank_property
            )
        else:
            docs = retrieve_function(query=query, collection=collection, top_k=top_k)
        
        formatted = "\n".join([
            f"Title:  {d['title']}, Chunk: {d['chunk'][:200]}..." 
            for d in docs
        ])
        
        prompt = f"Answer based on: {query}\nSources:\n{formatted}" if use_rag else query
        response = generate_with_single_input(prompt)
        return response['content']

print("llm_call đã được cập nhật để tương thích với display_widget!")
print("Dùng llm_call_advanced() khi cần dict output với metadata đầy đủ")

### 4.3 Specialized RAG Functions

In [ ]:
def fact_check(claim:  str, top_k: int = 10) -> dict:
    """
    Kiểm tra tính xác thực của một tuyên bố dựa trên news sources.
    """
    # Retrieve relevant documents
    documents = hybrid_retrieve(
        query=claim, 
        collection=collection, 
        alpha=0.3,
        top_k=top_k
    )
    
    # Format sources
    formatted_sources = ""
    for i, doc in enumerate(documents, 1):
        formatted_sources += f"""
[Source {i}]
Title: {doc['title']}
Content: {doc['chunk'][:300]}...
Date: {doc['pubDate']}
URL: {doc['link']}
"""
    
    PROMPT = f"""You are a fact-checker.  Analyze the following claim against the provided news sources. 

**Claim to verify**:  {claim}

**News Sources**:
{formatted_sources}

**Instructions**:
1. Carefully compare the claim with information in the sources
2. Determine if the claim is:  true, false, partially_true, or unverifiable
3. Provide specific evidence from the sources
4. Be concise

**Response Format** (use exactly this format):
VERDICT: [true/false/partially_true/unverifiable]
EVIDENCE: 
- [evidence point 1]
- [evidence point 2]
SOURCES:  [Source numbers used, e.g., 1, 3, 5]
"""
    
    response = generate_with_single_input(PROMPT)
    content = response['content']
    
    # Parse response manually instead of expecting JSON
    result = {
        "claim": claim,
        "verdict": "unverifiable",
        "evidence": [],
        "sources":  [],
        "raw_response": content
    }
    
    # Try to extract verdict
    lines = content.split('\n')
    for line in lines: 
        line_lower = line.lower().strip()
        if line_lower.startswith('verdict:'):
            verdict_text = line_lower. replace('verdict:', '').strip()
            if 'true' in verdict_text and 'partially' not in verdict_text:
                result['verdict'] = 'true'
            elif 'false' in verdict_text: 
                result['verdict'] = 'false'
            elif 'partially' in verdict_text: 
                result['verdict'] = 'partially_true'
            else:
                result['verdict'] = 'unverifiable'
        elif line. strip().startswith('- '):
            result['evidence'].append(line.strip()[2:])
    
    return result


def compare_events(event1: str, event2: str, top_k:  int = 5) -> str:
    """
    So sánh hai sự kiện dựa trên news coverage.
    
    Args:
        event1: Sự kiện thứ nhất
        event2: Sự kiện thứ hai
        top_k:  Số sources cho mỗi sự kiện
        
    Returns:
        str: Phân tích so sánh
    """
    # Retrieve documents cho cả hai events
    docs1 = semantic_search_retrieve(query=event1, collection=collection, top_k=top_k)
    docs2 = semantic_search_retrieve(query=event2, collection=collection, top_k=top_k)
    
    # Format sources
    formatted_event1 = "\n".join([
        f"- {doc['title']}: {doc['chunk'][: 200]}..." for doc in docs1
    ])
    formatted_event2 = "\n".join([
        f"- {doc['title']}: {doc['chunk'][:200]}..." for doc in docs2
    ])
    
    PROMPT = f"""Compare and contrast the following two events based on news coverage: 

**Event 1**: {event1}
Sources: 
{formatted_event1}

**Event 2**: {event2}
Sources: 
{formatted_event2}

**Provide**:
1. Summary of each event
2. Key similarities
3. Key differences
4. Relative media coverage/importance
5. Any connections between the events
"""
    
    response = generate_with_single_input(PROMPT, temperature=0.5)
    return response['content']


def summarize_topic(topic: str, time_range: str = "2024", top_k: int = 10) -> dict:
    """
    Tổng hợp tin tức về một chủ đề. 
    """
    # Retrieve documents
    documents = hybrid_retrieve(
        query=topic, 
        collection=collection, 
        alpha=0.6,
        top_k=top_k
    )
    
    # Format sources with dates
    formatted_sources = ""
    for doc in sorted(documents, key=lambda x: str(x['pubDate'])):
        formatted_sources += f"""
Date: {doc['pubDate']}
Title: {doc['title']}
Summary: {doc['chunk'][:200]}... 
---
"""
    
    PROMPT = f"""Create a comprehensive summary of news about "{topic}" in {time_range}. 

**Available News Articles**:
{formatted_sources}

**Provide your response in this format**: 

EXECUTIVE SUMMARY:
[2-3 sentence overview]

KEY POINTS:
- [point 1]
- [point 2]
- [point 3]

TIMELINE:
- [Date]:  [Event description]
- [Date]: [Event description]

TRENDS:
[Analysis of how the topic evolved]
"""
    
    response = generate_with_single_input(PROMPT)
    content = response['content']
    
    # Parse response manually
    result = {
        "topic": topic,
        "time_range": time_range,
        "executive_summary": "",
        "key_points": [],
        "timeline": [],
        "trends":  "",
        "raw_response": content
    }
    
    # Extract sections
    lines = content.split('\n')
    current_section = None
    
    for line in lines: 
        line_stripped = line.strip()
        line_upper = line_stripped. upper()
        
        if 'EXECUTIVE SUMMARY' in line_upper:
            current_section = 'summary'
        elif 'KEY POINTS' in line_upper:
            current_section = 'points'
        elif 'TIMELINE' in line_upper: 
            current_section = 'timeline'
        elif 'TRENDS' in line_upper:
            current_section = 'trends'
        elif line_stripped: 
            if current_section == 'summary' and not line_stripped.startswith('-'):
                result['executive_summary'] += line_stripped + ' '
            elif current_section == 'points' and line_stripped.startswith('-'):
                result['key_points']. append(line_stripped[1:]. strip())
            elif current_section == 'timeline' and line_stripped. startswith('-'):
                result['timeline']. append(line_stripped[1:].strip())
            elif current_section == 'trends' and not line_stripped.startswith('-'):
                result['trends'] += line_stripped + ' '
    
    result['executive_summary'] = result['executive_summary']. strip()
    result['trends'] = result['trends'].strip()
    
    return result

<a id='5'></a>
## 5 - Experimenting with Your RAG System

Now it is time for you to experiment with the system! Run the next cell to load a widget that will input a query, a rerank property, and output five different LLM responses:

1. With semantic search
2. With semantic search and reranking
3. With BM25 search
4. With hybrid search
5. Without RAG


In [ ]:
def enhanced_rag_demo():
    """
    Demo tất cả tính năng của enhanced RAG system. 
    """
    print("=" * 60)
    print("ENHANCED RAG SYSTEM DEMO")
    print("=" * 60)
    
    # Demo 1: Auto-tuned Technical Query (không dùng rerank)
    print("\nDemo 1: Auto-tuned Technical Query")
    print("-" * 40)
    query1 = "What was the GDP growth rate of the US in 2024?"
    try:
        result1 = llm_call_advanced(query1, auto_tune=True, output_format="text", use_rerank=False)
        print(f"Query: {query1}")
        print(f"Detected Type: {result1['query_type']}")
        print(f"Category:  {result1['category']}")
        print(f"Params Used: {result1['params_used']}")
        print(f"\nResponse:\n{result1['content'][: 500]}...")
    except Exception as e: 
        print(f"Error in Demo 1: {e}")
    
    # Demo 2: Creative query
    print("\n\nDemo 2: Auto-tuned Creative Query")
    print("-" * 40)
    query2 = "Analyze the political implications of US-Brazil relations in 2024"
    try:
        result2 = llm_call_advanced(query2, auto_tune=True, output_format="text", use_rerank=False)
        print(f"Query: {query2}")
        print(f"Detected Type: {result2['query_type']}")
        print(f"Category:  {result2['category']}")
        print(f"\nResponse:\n{result2['content'][:500]}...")
    except Exception as e:
        print(f"Error in Demo 2: {e}")
    
    # Demo 3: Với Reranking (sử dụng đúng function)
    print("\n\nDemo 3: Query với Reranking")
    print("-" * 40)
    query3 = "Tell me about Taylor Swift's concerts"
    try: 
        result3 = llm_call_advanced(
            query3, 
            auto_tune=False,  # Tắt auto_tune để control params
            use_rerank=True, 
            rerank_property='chunk',
            top_k=5,
            output_format="text"
        )
        print(f"Query: {query3}")
        print(f"Using Reranking:  True")
        print(f"\nResponse:\n{result3['content'][:500]}...")
    except Exception as e:
        print(f"Error in Demo 3: {e}")
    
    # Demo 4: Fact checking
    print("\n\nDemo 4: Fact Checking")
    print("-" * 40)
    claim = "Taylor Swift broke the record at Wembley Stadium in 2024"
    try:
        result4 = fact_check(claim)
        print(f"Claim: {claim}")
        print(f"Verdict: {result4.get('verdict', 'N/A')}")
        evidence = result4.get('evidence', [])
        if evidence:
            print(f"Evidence:  {evidence[: 2]}")
    except Exception as e: 
        print(f"Error in Demo 4: {e}")
    
    # Demo 5: Topic Summary
    print("\n\nDemo 5: Topic Summary")
    print("-" * 40)
    try:
        result5 = summarize_topic("US Economy", "2024", top_k=5)
        print(f"Topic:  US Economy in 2024")
        print(f"Executive Summary: {result5. get('executive_summary', 'N/A')}")
        key_points = result5.get('key_points', [])
        if key_points:
            print(f"Key Points:  {key_points[: 3]}")
    except Exception as e: 
        print(f"Error in Demo 5: {e}")
    
    print("\n" + "=" * 60)
    print("DEMO COMPLETE")
    print("=" * 60)

In [ ]:
enhanced_rag_demo()

In [ ]:
display_widget(llm_call, semantic_search_retrieve, bm25_retrieve, hybrid_retrieve, semantic_search_with_reranking)